# Cricket Object Detection - Project Notebook

This notebook demonstrates the complete pipeline for detecting cricket objects (bat, ball, stumps) in images using an 8x8 grid-based approach with hand-crafted features.

## Problem Statement
- Detect whether regions in a cricket image contain a bat, ball, stumps, or no object
- Each 800x600 image is divided into an 8x8 grid (64 cells)
- Use hand-crafted features only (no CNNs)

## Classes
- 0 → No object
- 1 → Ball
- 2 → Bat
- 3 → Stump

In [ ]:
# Import required libraries
import sys
import os
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt

# Add source directory to path
sys.path.insert(0, 'src')

from utils import (
    CLASS_LABELS, TOTAL_CELLS, IMAGE_WIDTH, IMAGE_HEIGHT,
    GRID_ROWS, GRID_COLS, CELL_WIDTH, CELL_HEIGHT,
    get_grid_cell_bounds, create_csv_header, create_csv_row
)
from preprocess import preprocess_image, split_image_to_grid, image_to_numpy
from features import extract_grid_features, extract_cell_features
from annotate import create_annotation, save_annotations_batch, visualize_annotation
from train import train_model, prepare_training_data, save_trained_model
from predict import predict_image, generate_csv_output, load_trained_model

print("All modules imported successfully!")

## 1. Understanding the Grid System

Each 800x600 image is divided into an 8x8 grid:
- 64 cells total
- Each cell is 100x75 pixels
- Cells are numbered c01 to c64 (row-major order)

In [ ]:
# Visualize the grid system
print(f"Image dimensions: {IMAGE_WIDTH}x{IMAGE_HEIGHT}")
print(f"Grid size: {GRID_ROWS}x{GRID_COLS} = {TOTAL_CELLS} cells")
print(f"Cell size: {CELL_WIDTH}x{CELL_HEIGHT} pixels")

# Show grid cell numbering
print("\nGrid Cell Numbering (c01 to c64):")
for row in range(GRID_ROWS):
    cells = [f"c{row*GRID_COLS+col+1:02d}" for col in range(GRID_COLS)]
    print(" | ".join(cells))

In [ ]:
# Demonstrate cell bounds
print("Cell Bounds Examples:")
for cell_idx in [1, 8, 35, 64]:
    x1, y1, x2, y2 = get_grid_cell_bounds(cell_idx)
    print(f"  Cell c{cell_idx:02d}: x=[{x1}, {x2}), y=[{y1}, {y2})")

## 2. Image Preprocessing

Images must be:
- 4:3 aspect ratio
- 800x600 pixels
- Not upscaled (min original size 800x600)

In [ ]:
# Create a synthetic test image for demonstration
def create_synthetic_cricket_image():
    """Create a synthetic cricket image for testing."""
    img = np.ones((600, 800, 3), dtype=np.uint8) * 100  # Gray background
    
    # Add green grass-like texture
    for y in range(400, 600):
        intensity = 100 + np.random.randint(0, 50)
        img[y, :, 1] = intensity  # Green channel
    
    # Add red ball in cell 35 (row 4, col 3)
    cy, cx = 262, 250  # Center of cell 35
    for y in range(cy-20, cy+20):
        for x in range(cx-20, cx+20):
            if (y-cy)**2 + (x-cx)**2 < 20**2:
                img[y, x] = [220, 20, 20]  # Red
    
    # Add brown bat in cells 20, 55, 63 (diagonal)
    for cell_idx in [20, 55, 63]:
        x1, y1, x2, y2 = get_grid_cell_bounds(cell_idx)
        # Draw diagonal bat segment
        for i in range(75):
            if y1+i < 600 and x1+i < x2:
                img[y1+i, x1+10:x1+30] = [139, 90, 43]  # Brown
    
    # Add white stumps in cells 28, 36
    for cell_idx in [28, 36]:
        x1, y1, x2, y2 = get_grid_cell_bounds(cell_idx)
        # Draw vertical stumps
        img[y1+5:y2-5, x1+45:x1+55] = [240, 240, 240]  # White
    
    return img

# Create and display
synthetic_img = create_synthetic_cricket_image()
plt.figure(figsize=(12, 9))
plt.imshow(synthetic_img)
plt.title('Synthetic Cricket Image for Testing')
plt.axis('off')
plt.show()

## 3. Feature Extraction

We use hand-crafted features including:
- Color histograms (RGB and HSV)
- Histogram of Oriented Gradients (HOG)
- Edge features
- Shape features (Hu moments)
- Color moments
- Texture features

In [ ]:
# Split image into grid cells
cells = split_image_to_grid(synthetic_img)
print(f"Split image into {len(cells)} cells")
print(f"Each cell shape: {cells[0].shape}")

# Visualize some cells
fig, axes = plt.subplots(2, 4, figsize=(12, 6))
cells_to_show = [0, 19, 27, 34, 35, 54, 62, 63]  # Various cells

for ax, cell_idx in zip(axes.flat, cells_to_show):
    ax.imshow(cells[cell_idx])
    ax.set_title(f'Cell c{cell_idx+1:02d}')
    ax.axis('off')

plt.tight_layout()
plt.show()

In [ ]:
# Extract features from cells
print("Extracting features from all 64 cells...")
all_features = extract_grid_features(synthetic_img)

print(f"\nFeatures extracted:")
print(f"  Number of cells: {len(all_features)}")
print(f"  Features per cell: {len(all_features[0])}")

# Show feature vector structure
feature = all_features[34]  # Cell 35 (ball)
print(f"\nFeature breakdown (example from cell 35):")
print(f"  Total features: {len(feature)}")
print(f"  Feature min: {feature.min():.4f}")
print(f"  Feature max: {feature.max():.4f}")
print(f"  Feature mean: {feature.mean():.4f}")

## 4. Annotation

Create annotations for training data.

In [ ]:
# Create annotation for our synthetic image
# Based on the objects we placed:
# - Ball in cell 35
# - Bat in cells 20, 55, 63
# - Stumps in cells 28, 36

cell_labels = {
    35: 1,  # Ball
    20: 2,  # Bat
    55: 2,  # Bat
    63: 2,  # Bat
    28: 3,  # Stump
    36: 3,  # Stump
}

annotation = create_annotation('synthetic_test.jpg', cell_labels)

print("Annotation created:")
print(f"  Image: {annotation['image_filename']}")
print(f"  Labels: {annotation['labels']}")

# Summarize
for cls, name in CLASS_LABELS.items():
    count = annotation['labels'].count(cls)
    print(f"  {name}: {count} cells")

In [ ]:
# Visualize annotation on image
annotated_img = visualize_annotation(synthetic_img, annotation['labels'])

plt.figure(figsize=(12, 9))
plt.imshow(annotated_img)
plt.title('Annotated Image (Gray=0, Red=1, Green=2, Blue=3)')
plt.axis('off')
plt.show()

## 5. Training

Train the model using extracted features.

In [ ]:
# Prepare training data (using synthetic images)
# In practice, you would load real images and annotations

# Create multiple synthetic images with different object positions
np.random.seed(42)
n_train_images = 10

train_images = []
train_labels = []

print(f"Creating {n_train_images} synthetic training images...")

for i in range(n_train_images):
    # Create base image
    img = np.ones((600, 800, 3), dtype=np.uint8) * (80 + np.random.randint(40))
    
    # Add grass texture
    for y in range(400, 600):
        img[y, :, 1] = 100 + np.random.randint(0, 50)
    
    # Random labels
    labels = [0] * 64
    
    # Add random ball
    ball_cell = np.random.randint(1, 65)
    labels[ball_cell-1] = 1
    x1, y1, x2, y2 = get_grid_cell_bounds(ball_cell)
    cy, cx = (y1+y2)//2, (x1+x2)//2
    for y in range(max(0,cy-15), min(600,cy+15)):
        for x in range(max(0,cx-15), min(800,cx+15)):
            if (y-cy)**2 + (x-cx)**2 < 15**2:
                img[y, x] = [220, 20, 20]
    
    # Add random bat cells
    bat_cells = np.random.choice(range(1, 65), size=2, replace=False)
    for cell_idx in bat_cells:
        labels[cell_idx-1] = 2
        x1, y1, x2, y2 = get_grid_cell_bounds(cell_idx)
        img[y1+5:y2-5, x1+20:x1+40] = [139, 90, 43]
    
    # Add random stump
    stump_cell = np.random.randint(1, 65)
    labels[stump_cell-1] = 3
    x1, y1, x2, y2 = get_grid_cell_bounds(stump_cell)
    img[y1+5:y2-5, x1+45:x1+55] = [240, 240, 240]
    
    train_images.append(img)
    train_labels.append(labels)

print(f"Created {len(train_images)} training images")

In [ ]:
# Prepare features for training
X, y = prepare_training_data(train_images, train_labels)

print(f"Training data prepared:")
print(f"  Feature matrix shape: {X.shape}")
print(f"  Label vector shape: {y.shape}")

In [ ]:
# Train the model
print("Training Random Forest classifier...")
model, mean, std = train_model(X, y, model_type='random_forest')

# Save the model
os.makedirs('models', exist_ok=True)
model_path = save_trained_model(model, mean, std, 'PravinTeam', 'models')
print(f"\nModel saved to: {model_path}")

## 6. Prediction

Make predictions on new images.

In [ ]:
# Make prediction on the original synthetic image
# First, simulate loading from file by extracting features directly

cell_features = extract_grid_features(synthetic_img)
X_test = np.array(cell_features)
X_test_scaled = (X_test - mean) / std

predictions = model.predict(X_test_scaled)

print("Predictions for synthetic test image:")
print(f"\nGrid of predictions (8x8):")
for row in range(8):
    row_preds = predictions[row*8:(row+1)*8]
    print(f"Row {row+1}: {' '.join(str(int(p)) for p in row_preds)}")

# Count objects
print(f"\nObject counts:")
for cls, name in CLASS_LABELS.items():
    count = np.sum(predictions == cls)
    print(f"  {name}: {count}")

## 7. CSV Output Generation

In [ ]:
# Generate CSV output
header = create_csv_header()
print("CSV Header:")
print(header)

# Create a row for our prediction
row = create_csv_row('synthetic_test.jpg', 'Test', predictions.tolist())
print("\nCSV Row:")
print(row)

In [ ]:
# Write to CSV file
os.makedirs('outputs', exist_ok=True)
output_path = 'outputs/predictions.csv'

with open(output_path, 'w') as f:
    f.write(header + '\n')
    f.write(row + '\n')

print(f"CSV output written to: {output_path}")

# Display file contents
print("\nFile contents:")
with open(output_path, 'r') as f:
    print(f.read())

## Summary

This notebook demonstrated the complete pipeline:

1. **Grid System**: 800x600 images divided into 8x8 grid (64 cells of 100x75 pixels)

2. **Feature Extraction**: Hand-crafted features including:
   - Color histograms (RGB and HSV)
   - HOG (Histogram of Oriented Gradients)
   - Edge features
   - Shape features (Hu moments)
   - Color moments
   - Texture features

3. **Model Training**: Random Forest or Logistic Regression classifiers

4. **Prediction**: Per-cell classification (0=no_object, 1=ball, 2=bat, 3=stump)

5. **CSV Output**: Format: ImageFileName, TrainOrTest, c01, c02, ..., c64

### Next Steps for Real Implementation:
1. Collect 300+ cricket images
2. Preprocess images to 800x600
3. Create annotations for each image
4. Train the model
5. Evaluate on test set
6. Generate final CSV output